In [107]:
import pandas as pd
import numpy as np
import min_features, daily_return
import importlib
importlib.reload(min_features)
importlib.reload(daily_return)

perf_df = pd.read_csv("all_sets.csv")
perf_df['Date'] = perf_df['test_start']
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feats = daily_return.pull_daily('QQQ', returns) 
return_cols = df_daily.columns[df_daily.columns.str.contains("Return_")].to_list()

In [100]:
# feature_lists maps feature_set NAME -> list of column names
# top_models has columns: ["horizon", "feature_set"]

min_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]

if min_feats != 'N':
    df_min = min_features.min_features()
    df_daily, feature_sets = daily_return.pull_daily('QQQ', returns) 

    df_main = pd.merge(df_min, df_daily, how='inner', on='Date')
    df_main = df_main.sort_values(by='Date', ascending=False)

    return_cols = df_main.columns[df_main.columns.str.contains("Return_")].to_list()
    daily_cols = [
        c for c in df_daily.iloc[:, 1:].columns
        if "return" not in c.lower()
    ]
    close_cols = df_min.columns[(df_min.columns.str.contains("close_")) | (df_min.columns.str.contains("post_")) | (df_min.columns.str.contains("overnight_"))].to_list()
    min_cols = (
        df_min
        .loc[:, ~df_min.columns.isin(close_cols)]  # drop close_ columns
        .iloc[:, 1:]                               # drop first column
        .columns
        .to_list()
    )
else:
    df_daily, feature_sets = daily_return.pull_daily('QQQ', returns) 
    return_cols = df_daily.columns[df_daily.columns.str.contains("Return_")].to_list()
    daily_cols = [
        c for c in df_daily.iloc[:, 1:].columns
        if "return" not in c.lower()
    ]
    df_main = df_daily[df_daily['Date'] <= '2026-01-21'].copy()

top_models = pd.read_csv("top_performers2.csv")
print(f'Available Feature Sets: {feature_sets.keys()}')

ma_all_cols = feature_sets['ma']
rsi_cols = feature_sets['rsi']
macd_cols = feature_sets['macd']
volu_cols = feature_sets['volume']
atr_adx_cols = feature_sets['atr_adx']
vola_cols = feature_sets['volatility']
#ma_no_days_cols = [c for c in ma_all_cols if not c.startswith("num_days")]

ma_no_lag = [c for c in ma_all_cols if "lag_" not in c.lower()]
ma_lag = [c for c in ma_all_cols if "rel_" not in c.lower()]
ma_no_rel = [c for c in ma_lag if "rel_" not in c.lower()]
ma_no_rel_rsi = ma_no_rel + rsi_cols
ma_lag_rsi = ma_lag + rsi_cols
ma_no_rel_macd = ma_no_rel + macd_cols
ma_lag_rsi_macd = ma_lag_rsi + macd_cols
ma_no_rel_volu = ma_no_rel + volu_cols
ma_no_rel_aa = ma_no_rel + atr_adx_cols
ma_lag_aa = ma_lag + atr_adx_cols

feature_lists = {
    "ma_lag-rsi_macd": ma_lag_rsi_macd,
    "ma_no_rel_aa_volu": ma_no_rel_aa + volu_cols,
}

# build: horizon -> { feature_set_name -> columns_list }
horizon_feature_cols = {
    r: {
        fs: feature_lists[fs]
        for fs in top_models.loc[top_models["horizon"] == r, "feature_set"].unique()
        if fs in feature_lists
    }
    for r in top_models["horizon"].unique()
}

# (optional) quick validation print
for r, fs_map in horizon_feature_cols.items():
    print(f"\nHorizon {r}")
    for fs_name, cols in fs_map.items():
        print(f"  {fs_name}: {len(cols)} cols")


Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])

Horizon 2
  ma_lag-rsi_macd: 100 cols
  ma_no_rel_aa_volu: 116 cols

Horizon 5
  ma_lag-rsi_macd: 100 cols
  ma_no_rel_aa_volu: 116 cols

Horizon 10
  ma_lag-rsi_macd: 100 cols
  ma_no_rel_aa_volu: 116 cols

Horizon 20
  ma_lag-rsi_macd: 100 cols
  ma_no_rel_aa_volu: 116 cols

Horizon 30
  ma_lag-rsi_macd: 100 cols
  ma_no_rel_aa_volu: 116 cols


In [114]:
def flip_bucket_tables_multi_dual(
    df_daily,
    perf_df,
    returns,
    *,
    K=3,
    date_col="Date",
    close_col="Close",
    w=None,
    perf_filter=None,  # optional callable to filter perf_df per horizon
):
    """
    For each horizon r:
      - builds streak + streak_lag1 from Return_r
      - merges into perf_df rows for (horizon=r) (and any extra filters you provide)
      - buckets BOTH streak and streak_lag1 into [-K..K] plus tails as +/- (K+1) labeled "3+"
      - computes:
          - wba_close (from streak) and wba_open (from streak_lag1)
          - bal_acc pair scores for +/-1, +/-2, +/-3 for both contexts
          - (optional) keeps acc/n wide columns for each context with suffixes _c and _o

    Returns:
      by_r: dict[r] -> wide table (flat columns)
      all_out: concat of all horizons with horizon as index level (flat columns)
    """
    if w is None:
        # weights: ±1 -> 2.0, ±2 -> 1.5, ±3 -> 1.25, ±3+ -> 1.0
        w = {1: 2.0, 2: 1.5, 3: 1.25, "3+": 1.0}

    max_score = float(sum(w.values()))
    gcols = ["model", "train_years", "feature_set"]

    def _add_streak(df_base, ret_col):
        d = df_base[[date_col, close_col, ret_col]].sort_values(date_col).copy()
        s = d[ret_col].astype("int8")
        grp = s.ne(s.shift()).cumsum()
        streak_len = s.groupby(grp).cumcount() + 1
        d["streak"] = streak_len.where(s.eq(1), -streak_len).astype("int32")
        d["streak_lag1"] = d["streak"].shift(1).fillna(0).astype("Int64")
        return d

    def _bucketize(series: pd.Series) -> pd.Series:
        b = series.clip(lower=-K, upper=K).astype("int32")
        b = b.copy()
        b.loc[series < -K] = -(K + 1)
        b.loc[series >  K] =  (K + 1)
        return b

    def _make_context_table(d: pd.DataFrame, bucket_col: str, suffix: str) -> pd.DataFrame:
        """
        suffix: "_c" for streak (close), "_o" for streak_lag1 (open)
        returns a flat-column wide df indexed by gcols, containing:
          - acc_{bucket}{suffix}, n_{bucket}{suffix} for buckets {1,-1,2,-2,3,-3,3+,-3+}
          - bal_acc_{1/2/3}{suffix}
          - wba_{close/open} (handled outside)
        """
        flip_perf = (
            d.groupby(gcols + [bucket_col], sort=False)
             .agg(n=("acc", "size"), acc=("acc", "mean"))
        )

        wide = pd.concat(
            {"acc": flip_perf["acc"].unstack(bucket_col),
             "n":   flip_perf["n"].unstack(bucket_col)},
            axis=1
        )

        # enforce order: +1/-1, +2/-2, +3/-3, 3+/-3+
        ordered_cols = []
        for k in [1, 2, 3, "3+"]:
            pb = (K + 1) if k == "3+" else k
            nb = -(K + 1) if k == "3+" else -k
            ordered_cols += [("acc", pb), ("acc", nb), ("n", pb), ("n", nb)]
        wide = wide.reindex(columns=pd.MultiIndex.from_tuples(ordered_cols))

        # relabel bucket keys to strings ("3+", "-3+", etc.)
        rename_cols = []
        for metric, b in wide.columns:
            if b == (K + 1): lab = "3+"
            elif b == -(K + 1): lab = "-3+"
            else: lab = str(b)
            rename_cols.append((metric, lab))
        wide.columns = pd.MultiIndex.from_tuples(rename_cols)

        # flatten to single-level names: acc_1_c, n_-2_o, etc.
        flat = wide.copy()
        flat.columns = [f"{m}_{b}{suffix}" for (m, b) in flat.columns]

        # pair bal_acc for +/-1,2,3 (no 3+ requested here)
        def _pair(acc_pos, acc_neg):
            return (acc_pos + acc_neg) / 2

        flat[f"bal_acc_1{suffix}"] = _pair(flat.get(f"acc_1{suffix}"),  flat.get(f"acc_-1{suffix}"))
        flat[f"bal_acc_2{suffix}"] = _pair(flat.get(f"acc_2{suffix}"),  flat.get(f"acc_-2{suffix}"))
        flat[f"bal_acc_3{suffix}"] = _pair(flat.get(f"acc_3{suffix}"),  flat.get(f"acc_-3{suffix}"))
        flat[f"bal_acc_3p{suffix}"] = _pair(flat.get(f"acc_3+{suffix}"), flat.get(f"acc_-3+{suffix}"))

        return flat

    base = df_daily[[date_col, close_col] + [f"Return_{r}" for r in returns]].copy()

    by_r = {}

    for r in returns:
        ret_col = f"Return_{r}"
        df_r = _add_streak(base, ret_col)

        # merge with perf
        perf_r = perf_df[perf_df["horizon"] == r]
        if perf_filter is not None:
            perf_r = perf_filter(perf_r)

        d = df_r.merge(perf_r, on=date_col, how="inner")

        other = perf_r.copy()
        # bucket both contexts
        d["bucket_c"] = _bucketize(d["streak"])       # close-context
        d["bucket_o"] = _bucketize(d["streak_lag1"])  # open-context
        
        #other = d[['Date', "bucket_c"]]

        tab_c = _make_context_table(d, "bucket_c", "_c")
        tab_o = _make_context_table(d, "bucket_o", "_o")

        # combine side-by-side
        out = tab_c.join(tab_o, how="outer")

        # compute wba_close / wba_open from each context’s pair balances
        out["wba_close"] = (
            w[1]   * out["bal_acc_1_c"] +
            w[2]   * out["bal_acc_2_c"] +
            w[3]   * out["bal_acc_3_c"] +
            w["3+"] * out["bal_acc_3p_c"]
        ) / max_score

        out["wba_open"] = (
            w[1]   * out["bal_acc_1_o"] +
            w[2]   * out["bal_acc_2_o"] +
            w[3]   * out["bal_acc_3_o"] +
            w["3+"] * out["bal_acc_3p_o"]
        ) / max_score

        out["wba_close"] = out["wba_close"].round(2)
        out["wba_open"]  = out["wba_open"].round(2)

        # optional: keep horizon as a column too (handy for later concat)
        out = out.reset_index()
        out.insert(0, "horizon", r)

        by_r[r] = out

    all_out = pd.concat(by_r.values(), ignore_index=True)
    return by_r, all_out, other

# ---- usage ----
returns = [2, 5, 10, 20, 30]
by_r, all_out, d = flip_bucket_tables_multi_dual(
    df_daily=df_daily,
    perf_df=perf_df,
    returns=returns,
    K=3,
)

# horizon 10 table
by_r[10].sort_values(["wba_close", "wba_open"], ascending=False).head(25)

# all horizons combined
perf_columns = ['horizon', 'model', 'train_years', 'feature_set', 'n_1_c', 'acc_1_c', 'n_-1_c', 'acc_-1_c',
                 'bal_acc_1_o', 'bal_acc_2_o', 'bal_acc_3_o', 'bal_acc_3p_o', 'wba_open', 'wba_close']

# top per horizon (ranked by MCC desc, then Brier asc)
top_by_horizon = (
    all_out
    .sort_values(["horizon", 'wba_close', 'wba_open'], ascending=[True, False, False])
    .groupby("horizon", as_index=False, sort=False)
    .head(50)
)

# top 2 per horizon + anything containing "-rsi" in feature_set
top2 = (
    all_out
    .sort_values(["horizon", "wba_open", "wba_close"], ascending=[True, False, False])
    .groupby("horizon", sort=False)
    .head(6)
)
"""
rsi = all_out[all_out["feature_set"].astype(str).str.contains(r"-rsi", case=False, na=False)]

top_by_horizon = (
    pd.concat([top2, rsi], ignore_index=False)
      .drop_duplicates(subset=["horizon", "feature_set"], keep="first")
      .sort_values(["horizon", "wba_close", "wba_open"], ascending=[True, False, False])
      .reset_index(drop=True)
)

"""
top_by_horizon[perf_columns].round(2)

,horizon,model,train_years,feature_set,n_1_c,acc_1_c,n_-1_c,acc_-1_c,bal_acc_1_o,bal_acc_2_o,bal_acc_3_o,bal_acc_3p_o,wba_open,wba_close
8,2,random_forest,5,volu,35.0,0.69,35.0,0.29,0.61,0.51,0.53,0.52,0.55,0.52
0,2,random_forest,5,atr_adxvola,35.0,0.74,35.0,0.17,0.59,0.52,0.51,0.53,0.54,0.52
17,2,xgboost,5,volu,35.0,0.60,35.0,0.46,0.56,0.48,0.59,0.53,0.54,0.52
6,2,random_forest,5,rsi_macd,35.0,0.74,35.0,0.29,0.58,0.57,0.47,0.45,0.53,0.50
10,2,xgboost,5,experimental_slope,35.0,0.69,35.0,0.46,0.55,0.44,0.53,0.57,0.52,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,30,random_forest,5,ma_lag,6.0,0.17,7.0,0.86,0.38,0.45,0.58,0.46,0.46,0.43
70,30,xgboost,5,vix_skew,6.0,0.83,7.0,0.00,0.48,0.38,0.50,0.41,0.45,0.42
67,30,xgboost,5,ma_rel,6.0,0.17,7.0,0.57,0.55,0.28,0.54,0.44,0.46,0.40
58,30,random_forest,5,ma_rel,6.0,0.17,7.0,0.57,0.46,0.28,0.42,0.39,0.39,0.37


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, log_loss, matthews_corrcoef, balanced_accuracy_score

gcols = ["horizon", "model", "train_years", "feature_set"]

y_col = "test_pos_n"   # 0/1 actual
p_col = "pred"         # P(y=1)

def _clip01(p, eps=1e-15):
    p = np.asarray(p, dtype=float)
    return np.clip(p, eps, 1 - eps)

def _metrics(g: pd.DataFrame) -> pd.Series:
    y = g[y_col].astype(int).to_numpy()
    p = _clip01(g[p_col].to_numpy())

    # hard preds at 0.5
    yhat = (p >= 0.5).astype(int)

    # confident subset mask
    sel = (p >= 0.6) | (p <= 0.4)

    # probability metrics
    brier = brier_score_loss(y, p)
    ll = log_loss(y, p)

    # MCC (needs both classes in y and yhat)
    mcc = np.nan
    if (np.unique(y).size > 1) and (np.unique(yhat).size > 1):
        mcc = matthews_corrcoef(y, yhat)

    # Balanced accuracy (needs both classes in y)
    bal_acc = np.nan
    if np.unique(y).size > 1:
        bal_acc = balanced_accuracy_score(y, yhat)

    # confident accuracy + coverage
    cov = float(sel.mean())
    acc_conf = float((yhat[sel] == y[sel]).mean()) if sel.any() else np.nan

    return pd.Series({
        "pos_rate": float(y.mean()),
        "bal_acc": float(bal_acc) if not np.isnan(bal_acc) else np.nan,
        "brier": float(brier),
        "log_loss": float(ll),
        "mcc": float(mcc) if not np.isnan(mcc) else np.nan,
        "cov_|0.6|": cov,
        "acc_|0.6|": acc_conf,
    })

metrics_df = (
    perf_df
    .dropna(subset=[y_col, p_col])
    .groupby(gcols, sort=False)
    .apply(_metrics, include_groups=False)
    .reset_index()
    .sort_values(["horizon", "mcc", "brier"], ascending=[True, False, True])
)

metrics_df['composite'] = 0.5*(metrics_df['bal_acc']) + 0.25*(metrics_df['mcc']) + .125*(1-(metrics_df['brier'])) + 0.1*(metrics_df['acc_|0.6|']) - (1-(metrics_df['cov_|0.6|']))
# top per horizon (ranked by MCC desc, then Brier asc)
top_by_horizon = (
    metrics_df
    .sort_values(["horizon", "composite", "log_loss"], ascending=[True, False, False])
    .groupby("horizon", as_index=False, sort=False)
    .head(3)
)

#top_by_horizon[['horizon', 'feature_set']].to_csv('top_performers.csv', index=False)
top_by_horizon.round(2)

,horizon,model,train_years,feature_set,pos_rate,bal_acc,brier,log_loss,mcc,cov_|0.6|,acc_|0.6|,composite
0,2,xgboost,5,ma_lag,0.569565,0.519932,0.369391,3.588157,0.039969,0.913043,0.519048,0.313733
16,2,xgboost,5,experimental_slope,0.569565,0.516462,0.365913,4.524742,0.034545,0.926087,0.539906,0.326206
9,2,random_forest,5,rsi_macd,0.569565,0.509291,0.265620,0.730046,0.022149,0.647826,0.530201,0.052827
10,2,xgboost,5,volu,0.569565,0.504781,0.359870,3.663870,0.009630,0.926087,0.530516,0.313953
11,2,random_forest,5,volu,0.569565,0.502892,0.256772,0.707895,0.006713,0.608696,0.528571,0.007580
...,...,...,...,...,...,...,...,...,...,...,...,...
54,30,xgboost,5,ma_lag,0.739130,0.470098,0.380109,5.935268,-0.054680,0.947826,0.545872,0.301279
56,30,xgboost,5,ma_rel,0.739130,0.450000,0.481391,11.468517,-0.087942,0.973913,0.468750,0.288629
71,30,random_forest,5,experimental_slope,0.739130,0.441667,0.241565,0.679825,-0.136521,0.834783,0.619792,0.178269
57,30,random_forest,5,ma_rel,0.739130,0.408333,0.347467,0.963860,-0.161748,0.817391,0.425532,0.105241


In [19]:
def calibration_table(g, n_bins=8):
    y = g[y_col].astype(int).to_numpy()
    p = np.clip(g[p_col].to_numpy(dtype=float), 1e-15, 1-1e-15)

    # quantile bins (stable counts); duplicates="drop" prevents errors if probs repeat
    bins = pd.qcut(p, q=n_bins, duplicates="drop")

    out = (
        pd.DataFrame({"bin": bins, "y": y, "p": p})
        .groupby("bin", observed=True)
        .agg(
            n=("y", "size"),
            p_mean=("p", "mean"),   # predicted probability avg
            y_rate=("y", "mean"),   # observed frequency
            p_min=("p", "min"),
            p_max=("p", "max"),
        )
        .reset_index(drop=True)
        .sort_values("p_mean")
    )
    return out

cols = ['horizon']
# example: get calibration table for ONE group
key = (2)  # change
g = perf_df.set_index(cols).loc[key].reset_index()
calib_tbl = calibration_table(g, n_bins=10)
round(calib_tbl,2)

,n,p_mean,y_rate,p_min,p_max
0,317,0.09,0.43,0.00,0.20
1,334,0.35,0.39,0.25,0.40
2,184,0.45,0.46,0.45,0.45
3,447,0.52,0.53,0.50,0.55
4,235,0.60,0.69,0.60,0.60
5,228,0.65,0.66,0.65,0.65
6,194,0.70,0.67,0.70,0.70
7,272,0.79,0.71,0.75,0.85
8,525,0.97,0.71,0.90,1.00


In [ ]:
def calibration_table(g, n_bins=8):
    y = g[y_col].astype(int).to_numpy()
    p = np.clip(g[p_col].to_numpy(dtype=float), 1e-15, 1-1e-15)

    # quantile bins (stable counts); duplicates="drop" prevents errors if probs repeat
    bins = pd.qcut(p, q=n_bins, duplicates="drop")

    out = (
        pd.DataFrame({"bin": bins, "y": y, "p": p})
        .groupby("bin", observed=True)
        .agg(
            n=("y", "size"),
            p_mean=("p", "mean"),   # predicted probability avg
            y_rate=("y", "mean"),   # observed frequency
            p_min=("p", "min"),
            p_max=("p", "max"),
        )
        .reset_index(drop=True)
        .sort_values("p_mean")
    )
    return out

# example: get calibration table for ONE group
#key = (10, "xgboost", 6, "daily")  # change
g = perf_df.set_index(gcols).loc[key].reset_index()
calib_tbl = calibration_table(g, n_bins=20)
round(calib_tbl,2)